<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-модели-для-каждого-региона" data-toc-modified-id="Обучение-модели-для-каждого-региона-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение модели для каждого региона</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Выбор локации для скважины

Для компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Мне предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Так же проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
# загружаем необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# загружаем файлы
try:
    df1 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    df1 = pd.read_csv('/Users/dariasokolovska/Downloads/geo_data_0.csv')
    
try:
    df2 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    df2 = pd.read_csv('/Users/dariasokolovska/Downloads/geo_data_1.csv')
    
try:
    df3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df3 = pd.read_csv('/Users/dariasokolovska/Downloads/geo_data_2.csv')

Посмотрим как выглядят таблицы

In [3]:
display(df1.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
display(df2.head())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
display(df3.head())

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Изучим детальнее данные в датафреймах

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
display(df1.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
display(df2.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
display(df3.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Также необходимо выполнить проверку на наличие дубликатов

In [12]:
print("Количесвто дубликатов в df1:",df1.duplicated().sum())

Количесвто дубликатов в df1: 0


In [13]:
print("Количесвто дубликатов в df2:",df2.duplicated().sum())

Количесвто дубликатов в df2: 0


In [14]:
print("Количесвто дубликатов в df3:",df3.duplicated().sum())

Количесвто дубликатов в df3: 0


### Вывод

Изначально данные не имели дубликатов или пропусков, записаны в нужных форматах.
Поэтому готовы к последуюещему обучению.

## Обучение модели для каждого региона

Создадим функцию, в которой обучим и проверим модель для каждого региона

In [15]:
def prediction (df):
    features = df.drop(['product','id'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=1)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid = pd.Series(predicted_valid)

    rmse = (mean_squared_error(predicted_valid, target_valid))**(0.5)
    average_product = sum(predicted_valid) / len(predicted_valid)
    
    print("RMSE: {0:.2f}".format(rmse))
    print("Cредний запас предсказанного сырья: {0:.2f}".format(average_product))
    
    return (average_product, predicted_valid, target_valid.reset_index(drop=True), rmse)


In [16]:
average_product_1, predictions_1, target_1, rmse_1 = prediction(df1)
average_product_2, predictions_2, target_2, rmse_2 = prediction(df2)
average_product_3, predictions_3, target_3, rmse_3 = prediction(df3)

RMSE: 37.74
Cредний запас предсказанного сырья: 92.49
RMSE: 0.89
Cредний запас предсказанного сырья: 69.12
RMSE: 39.87
Cредний запас предсказанного сырья: 94.96


### Вывод

Лучше всего модель предсказала запасы во втором регионе (RMSE - 0.89), далее в первом и третьем соотвественно. При этом средний запас сырья больше в третьем регионе, и меньше в первом регионе.

##  Подготовка к расчёту прибыли

In [17]:
# сохраняем в отдельных переменных ключевые значения для расчётов 
budget = 10**10 
income_per_unit = 450 * 10**3
chosen_wells = 500
best_wells = 200

In [18]:
break_even_point = budget / income_per_unit
break_even_point_per_well = break_even_point/200

Создадим функцию, в которой сравним полученный объём сырья со средним запасом в каждом регионе

In [19]:
def comparison (average_product, break_even_point_per_well):
    if average_product > break_even_point_per_well:
        print('В регионе средний запас сырья выше чем обьем сырья для безубыточности на', average_product - break_even_point_per_well, 'тысячей баррелей')
    else:
        print('В регионе средний запас сырья ниже чем обьем сырья для безубыточности', break_even_point_per_well - average_product, 'тысячей баррелей')

In [20]:
comparison (average_product_1, break_even_point_per_well)

В регионе средний запас сырья ниже чем обьем сырья для безубыточности 18.618486512722683 тысячей баррелей


In [21]:
comparison (average_product_2, break_even_point_per_well)

В регионе средний запас сырья ниже чем обьем сырья для безубыточности 41.99070586825529 тысячей баррелей


In [22]:
comparison (average_product_3, break_even_point_per_well)

В регионе средний запас сырья ниже чем обьем сырья для безубыточности 16.154280625257954 тысячей баррелей


### Вывод 

В результате рассчетов я определила, что во всех регионах в среднем недостаточно сырья для безубыточности. Во втором регионе тот показатель самый высокий, а 1 и 3 примерно на одном уровне. 

## Расчёт прибыли и рисков

Создадим функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [23]:
def profit(target, predictions, count):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    p = income_per_unit * selected.sum() - budget
    return p


Создадим функцию для расчёта рисков и прибыли для каждого региона

In [24]:
def bootstrap (target, predictions, best_wells, profit):
    state = np.random.RandomState(12345)
    loss = 0
    values = []
    for i in range(1000):
        target_subsample = target.sample(500, replace = True, random_state = state)
        probs_subsample = predictions[target_subsample.index]
        prof = profit(target_subsample, probs_subsample, best_wells)
        values.append(prof)
        if prof < 0:
            loss +=1
    valuesSeries = pd.Series(values)
    lower = valuesSeries.quantile(0.025)
    upper = valuesSeries.quantile(0.975)
    
    
    loss = loss / 1000 * 100
    mean = valuesSeries.mean()
    print('Риск убытка:', loss, '%')
    print('Средняя прибыль:', mean)
    print('Доверительный интервал:', 'от',lower, 'до', upper)

In [25]:
bootstrap(target_1, predictions_1, best_wells, profit)

Риск убытка: 4.6 %
Средняя прибыль: 458100401.7997511
Доверительный интервал: от -91938453.32589239 до 1015824888.6825125


In [26]:
bootstrap(target_2, predictions_2, best_wells, profit)

Риск убытка: 0.1 %
Средняя прибыль: 533609292.92167586
Доверительный интервал: от 98001407.23450962 до 966461912.599809


In [27]:
bootstrap(target_3, predictions_3, best_wells, profit)

Риск убытка: 6.1 %
Средняя прибыль: 426348386.1666254
Доверительный интервал: от -116622627.7963096 до 980501749.734359


## Вывод

При выполнении проекта были выпонены следующие дествия:

- Изучины исходные данные, проведена подготовка данных к обучению.
- Модель обучена для каждого региона geo_data_0, geo_data_1 и geo_data_2.
- Создана функция расчета прибыли в регионе.
- Посчитана прибыль и риски для каждого регион.
Второй регион получился одним из лучших по показателям риска убытка (0.1%) и средней принбыли (533609292.92 руб.), хотя не все показатели данного региона были высокими.